In [2]:
import requests
from bs4 import BeautifulSoup
import re
import json
from urllib.parse import urljoin
from multiprocessing import Process, Manager

In [3]:
def extractEmailsFromPage(url, email_list):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    emails = soup.find_all('a', {"class": "__cf_email__"})
    decoded_emails = []
    
    for i in range(len(emails)):
        encoded = emails[i]['data-cfemail']
        decoded = cfDecodeEmail(encoded)
        decoded_emails.append(decoded)

    if len(decoded_emails) != 0:
        print(f"Emails found in url: {url}")

    email_list.extend(decoded_emails)

In [4]:
def cfDecodeEmail(encodedString):
    r = int(encodedString[:2],16)
    email = ''.join([chr(int(encodedString[i:i+2], 16) ^ r) for i in range(2, len(encodedString), 2)])
    return email

In [15]:
def extractEmailsFromLinks(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    emails = []

    for link in links:
        if 'href' in link.attrs:
            if link['href'].find("mailto:") == -1:
                continue
            else:
                email = link['href'].removeprefix("mailto:")
                emails.append(email)
        else:
            continue

    return emails


In [16]:
def getLinks(base_url, visited_list, url_list):
    response = requests.get(url=base_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')

    found_links = []

    for link in links:
        if 'href' in link.attrs:
                found_links.append(urljoin(base_url, link['href']))
        else:
            continue

    for link in found_links:
         if link not in visited_list and link not in url_list:
              url_list.append(link)

In [17]:
def scraper(base_url, limit, manager_dict):

    email_list = manager_dict['email_list']
    visited_list = manager_dict['visited_list']
    url_list = manager_dict['url_list']

    url_list.append(base_url)

    while url_list and len(email_list) < limit:
        curr_url = url_list.pop(0)
        if curr_url in visited_list:
            continue

        visited_list.append(curr_url)
        getLinks(curr_url, visited_list, url_list)

        print(f"Current url: {curr_url}")
        email_list.extend(extractEmailsFromPage(curr_url, email_list))

    manager_dict['email_list'] = email_list
    manager_dict['visited_list'] = visited_list
    manager_dict['url_list'] = url_list


In [18]:
if __name__ == "__main__":

    base_url = "https://www.dlsu.edu.ph/"
    limit = 10

    manager = Manager()
    manager_dict = manager.dict()
    manager_dict['email_list'] = manager.list()
    manager_dict['visited_list'] = manager.list()
    manager_dict['url_list'] = manager.list()

    processes = []

    for i in range(4):
        process = Process(target=scraper, args=(base_url, limit, manager_dict,))
        process.start()
        processes.append(process)
    
    for p in processes:
        p.join()

    print(manager_dict['url_list'])


2024-03-03 20:19:30,467 - INFO - Visited list: []


In [12]:
url = "https://www.dlsu.edu.ph/cdn-cgi/l/email-protection#aecbdac6cbc280c1c0c9eecac2dddb80cbcadb80dec6"
url = url.removeprefix("https://www.dlsu.edu.ph/cdn-cgi/l/email-protection#")
decoded = cfDecodeEmail(url)

if url.find("https://www.dlsu.edu.ph/cdn-cgi/l/email-protection#"):
    print(True)
else:
    print(False)

print(decoded)

True
ethel.ong@dlsu.edu.ph
